In [21]:
# большая ячейка с импортами
import random
import os
import string

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report


import requests as rq
from bs4 import BeautifulSoup

import telebot 
from telebot import types 

import random
import json
import re
from datetime import datetime
import calendar

# !pip install natasha
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,
    DatesExtractor,
    Doc
)
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)

#!pip install transliterate
from transliterate import translit

import markovify

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

## Обучение распознаванию интентов

Моей целью было создание бота, который может говорить о погоде (в любом городе, погоде текущей и на ближайшие дни), а также, совсем немного, помогать покупать книги.

Бот должен распознавать следующие интенты: приветствие, прощание, "как дела", благодарность, недовольство работой бота, а также намерение поговорить о погоде и литературе.

Я создала файл intents.json, в который вручную забила возможные паттерны (около 30 для каждого интента), а затем обучила на этих интентах классификатор.

Для классификатора я выбрала MLP Classifier, так как он показывал наивысшие результаты. Кроме него я пробовала логистическую регрессию, мультиноминального Байеса и случайные деревья.

In [22]:
with open(r'intents.json', encoding = 'utf-8') as file:
    data_str = file.read()

data_dict = json.loads(data_str)

phrases = []
classes_word = []
unique_classes = []

for intent in data_dict:
    for pattern in data_dict[intent]['patterns']:
        phrases.append(pattern)
        classes_word.append(intent)
    if intent not in unique_classes:
        unique_classes.append(intent)

indexed_classes =[]
for intent in classes_word:
    indexed_classes.append(unique_classes.index(intent))
    
vectorizer = CountVectorizer()
vectorizer.fit(phrases)
X_vec = vectorizer.transform(phrases)

X_train, X_test, y_train, y_test = train_test_split(X_vec,indexed_classes, test_size=0.25, random_state=42)

model = MLPClassifier()
model.fit(X_train, y_train)

C:\Users\marus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier()

In [23]:
y_preds = model.predict(X_test)
print(classification_report(y_preds, y_test))

              precision    recall  f1-score   support

           0       1.00      0.47      0.64        15
           1       0.80      1.00      0.89         4
           2       0.80      0.80      0.80        10
           3       0.92      1.00      0.96        11
           4       0.80      1.00      0.89         8
           5       0.67      1.00      0.80         6
           6       0.60      0.75      0.67         4

    accuracy                           0.81        58
   macro avg       0.80      0.86      0.81        58
weighted avg       0.85      0.81      0.80        58



В какой-то момент пришла идея дополнительно разграничивать интенты согласия и несогласия. В боте они используются только в одной функции, но потенциально их можно использовать и в других.

In [24]:
# Дополнительно-интенты да-нет

with open(r'my_intents.json', encoding = 'utf-8') as file:
    yes_no = file.read()

yes_no_dict = json.loads(yes_no)

yn_phrases = []
yn_classes_word = []
yn_unique_classes = []
yn_indexed_classes = []

for intent in yes_no_dict:
    for pattern in yes_no_dict[intent]['patterns']:
        yn_phrases.append(pattern)
        yn_classes_word.append(intent)
    if intent not in yn_unique_classes:
        yn_unique_classes.append(intent)

for intent in yn_classes_word:
    yn_indexed_classes.append(yn_unique_classes.index(intent))
    
yn_vectorizer = CountVectorizer()
yn_vectorizer.fit(yn_phrases)
yn_X_vec = yn_vectorizer.transform(yn_phrases)

yn_X_train, yn_X_test, yn_y_train, yn_y_test = train_test_split(yn_X_vec, yn_indexed_classes, test_size=0.25, random_state=42)

yn_model = MLPClassifier()
yn_model.fit(yn_X_train, yn_y_train)

yn_y_preds = yn_model.predict(yn_X_test)
print(classification_report(yn_y_preds, yn_y_test))

              precision    recall  f1-score   support

           0       0.86      0.67      0.75         9
           1       0.67      0.86      0.75         7

    accuracy                           0.75        16
   macro avg       0.76      0.76      0.75        16
weighted avg       0.77      0.75      0.75        16



C:\Users\marus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


## Функции, которые используются в боте

In [25]:
# api id для сайта open weather
api_id = 'c4f85f9f56687c24d9bc1396dd40f79a'

### Функции погодно-географические

В ячейках ниже будут функции (и некоторая другая информация), которые используются для выдачи информации о погоде

In [26]:
# Информация про время - сегодня, завтра и тд

# Список месяцев - они в усеченной форме, чтобы не лемматизировать входящий текст
months = ['январ', 'феврал', 'март', 'апрел', 'мая', 'июн', 'июл', 'август', 'сентябр', 'октябр', 'ноябр', 'декабр']


today = datetime.today()
today = today.replace(hour=0, minute = 0, second=0, microsecond =0)

# Удобнее ориентироваться, когда вся дата в одном формате - приводим все к timestamp
# Это также позволяет избежать проблемы с датами в конце/начале месяца
today_timestamp = int(today.timestamp())
tomorrow_timestamp = int(today_timestamp) + 86400
today_plus_two_timestamp = int(tomorrow_timestamp) + 86400
today_plus_three_timestamp = int(today_plus_two_timestamp) + 86400
today_plus_four_timestamp = int(today_plus_three_timestamp) + 86400
today_plus_five_timestamp = int(today_plus_four_timestamp) + 86400

# Здесь год, месяц и день в удобоваримом виде - чтобы выдавать текстовый аутпут
current_year = datetime.fromtimestamp(today_timestamp).year
current_month = datetime.fromtimestamp(today_timestamp).month
current_month_text = months[current_month - 1]
current_day = datetime.fromtimestamp(today_timestamp).day
good_days = [datetime.fromtimestamp(today_timestamp).day, datetime.fromtimestamp(tomorrow_timestamp).day,
            datetime.fromtimestamp(today_plus_two_timestamp).day, datetime.fromtimestamp(today_plus_three_timestamp).day, 
            datetime.fromtimestamp(today_plus_four_timestamp).day, datetime.fromtimestamp(today_plus_five_timestamp).day]

current_week_days = []
for i in range(5):
    current_week_days.append(calendar.weekday(today.year, today.month, today.day+i))
wkdays_names = ['понедельник','вторник', 'сред', 'четверг', 'пятниц', 'суббот', 'воскресенье']
good_weekdays = {wkdays_names[day] : i for i, day in enumerate(current_week_days)}

In [27]:
# Функция для нахождения координат по данному городу
def get_coords(place):
    city_info = rq.get(f"http://api.openweathermap.org/geo/1.0/direct?q={place}&appid={api_id}")
    if city_info.json() == []:
        return False
    
    else:
        latitude = city_info.json()[0]['lat']
        longitude = city_info.json()[0]['lon']
        return latitude, longitude

In [28]:
# Погода в данном городе вот прямо сейчас

def give_current_weather(place):
    latitude, longitude = get_coords(place)
    current_weather = rq.get(f"https://api.openweathermap.org/data/2.5/weather?lat={latitude}&lon={longitude}&appid={api_id}&units=metric&lang=ru")
    description = current_weather.json()['weather'][0]['description']
    temp = current_weather.json()['main']['temp']
    feels = current_weather.json()['main']['feels_like']
    weather_is = f'{place}: {description}, температура {temp}°C (ощущается как {feels}°C)'
    return weather_is

In [61]:
# Погода в данном городе на данное время

def give_weather_for_date(place, time):
    city_info = rq.get(f"http://api.openweathermap.org/geo/1.0/direct?q={place}&appid={api_id}")
    if city_info.json() == []:
        return 'Извини, не могу найти погоду для этого города :('
    else:
        lat = city_info.json()[0]['lat']
        lon = city_info.json()[0]['lon']
        future_weather = rq.get(f"http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={api_id}&units=metric&lang=ru")
        future_weather = future_weather.json()['list']

        # Даты выдаются в таймстемпах
        dates = [i['dt'] for i in future_weather]

        if time == today_timestamp:
            current_dates = [i for i, date in enumerate(dates) if date < tomorrow_timestamp]
            # Если запрашивать после 22:00, сайт время на сегодня не выдает - в таком случае перенаправляем на "время сейчас"
            if current_dates == []:
                return give_current_weather(place)
        elif time == tomorrow_timestamp:
            current_dates = [i for i, date in enumerate(dates) if tomorrow_timestamp <= date < today_plus_two_timestamp]
        elif time == today_plus_two_timestamp:
            current_dates = [i for i, date in enumerate(dates) if today_plus_two_timestamp <= date < today_plus_three_timestamp]
        elif time == today_plus_three_timestamp:
            current_dates = [i for i, date in enumerate(dates) if today_plus_three_timestamp <= date < today_plus_four_timestamp]
        elif time == today_plus_four_timestamp:
            current_dates = [i for i, date in enumerate(dates) if today_plus_four_timestamp <= date < today_plus_five_timestamp]
        else:
            current_dates = [i for i, date in enumerate(dates) if today_plus_five_timestamp <= date]


        answer = ''
        for i in current_dates:
            description = future_weather[i]['weather'][0]['description']
            temp = future_weather[i]['main']['temp']
            feels_like = future_weather[i]['main']['feels_like']
            when = datetime.fromtimestamp(future_weather[i]['dt']).time()
            answer += f'{when}: {description}, температура {temp}°C (ощущается как {feels_like}°C)\n'
            day = datetime.fromtimestamp(future_weather[i]['dt']).day
            month = datetime.fromtimestamp(future_weather[i]['dt']).month
        return f'Прогноз - {place} на {day}.{month}\n{answer}'

In [30]:
# Функции для даты

# По данным a=день, b=месяц выдает таймстемп 
def convert_date(a, b):
    date = int(datetime.strptime(f'{a}.{b}.{current_year}','%d.%m.%Y').timestamp())
    return date

# конвертирует дату, данную текстом, в таймстемп
def convert_text_date(time):
    if time == 'сегодня':
        return today_timestamp
    elif time == 'завтра':
        return tomorrow_timestamp
    elif time == 'послезавтра' or time == 'через два дня' or time == 'через 2 дня':
        return today_plus_two_timestamp
    elif time == 'через три дня' or time == 'через 3 дня':
        return today_plus_three_timestamp
    else:
        return today_plus_four_timestamp

# Выделяет дату из сообщения
# я пробовала использовать dates extractor из библиотеки natasha, но он плохо распознает случаи, когда дата в формате
# без года, а также совсем не распознает месяц в текстовом формате
# Регулярные выражения в свою очередь с этой задачей справляются отлично.
# Потенциально можно добавить определение числа из текста ("первое января")

def find_if_they_said_date(text):
    
    text_date = re.findall('сейчас|сегодня|завтра|послезавтра|через два дня|через три дня|через четыре дня|через 2 дня|через 3 дня|через 4 дня', text.lower())
    day_month = re.findall('\d{1,2}\.\d{1,2}', text)
    day_month_text = re.findall('\d{1,2}?\s[январ|феврал|март|апрел|мая|июн|июл|август|сентябр|октябр|ноябр|декабр]+', text.lower())
    text_weekday = re.findall('понедельник|вторник|сред|четверг|пятниц|суббот|воскресенье', text.lower())

    if text_date != []:
        if text_date[0] != 'сейчас':
            return convert_text_date(text_date[0])
        else:
            return 'сейчас'
        
    if text_weekday != [] and text_weekday[0] in good_weekdays.keys():
        weekday = good_weekdays[text_weekday[0]]
        a = good_days[weekday]
        b = current_month
        return convert_date(a, b)
        
    if day_month != []:
        for i in day_month:
            a, b = i.split('.')
            if int(b) == current_month and int(a) in good_days:
                return convert_date(a, b)
            return False

    elif day_month_text != []:
        for i in day_month_text:
            a, b = i.split(' ')
            if b[0:2] == current_month_text[0:2] and int(a) in good_days:
                b = current_month
                return convert_date(a, b)
            return False

    else:
        return False

In [31]:
# Вытаскиваем город из текста: names extractor из natasha
def get_city(text):
    city = ''
    doc = Doc(text.title())
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)
    for span in doc.spans:
        span.normalize(morph_vocab)
    for span in doc.spans:
        if span.type == 'LOC':
            span.extract_fact(names_extractor)
            city = span.normal
    if city != '':
        return city
    return False

In [32]:
# Маленькое дополнение - выяснилось, что open weather может выдавать прогноз на довольно необычные города :) 
# Их пришлось вынести в исключения - к сожалению, пользователь не сможет узнать погоду в городе "Погода", 
# но это небольшая плата за то, чтобы бот не считывал сообщение "Погода" как город 
fake_places = ['привет', 'погода', 'литература', "книга", "книги", "пока"]

### Функции книжные

In [33]:
# По данному названию книги/автора находит эту книгу в книжном магазине
def get_book(book_query):
    url = f"https://www.labirint.ru/search/{book_query}/?stype=0"
    page = rq.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    # Все книги на странице
    book_info = {}
    try:
        books = soup.findAll("div", class_ = "product-cover")
    
        book_info = ''
        # Ищем первые три
        for i in books[:3]:
            book_number = i.find("a").get('href')
            book_name = i.find("a").get("title")
            book_price = i.find("span", class_ = "price-val").text
            book_price = re.sub('\t', '', book_price)
            book_price = re.sub("\n", '', book_price)
            book_price = re.sub('\r', '', book_price)
            book_info += f"- {book_name} ({book_price}): https://www.labirint.ru{book_number}\n"
        return book_info
    except:
        return False

In [34]:
# Функция для выделения имен из текста - для выделения имени автора
# Изначально пыталась использовать только names extractor из natasha, но он работает неидеально
# Например, Пушкина он не распознавал. Поэтому я решила спарсить имена известных авторов и попытаться сначала пробежаться по ним

# Файл с парсингом лежит в этом же репозитории
with open(r'authors_surnames.txt') as file:
    authors_surnames = file.read()
    
authors_surnames = authors_surnames[0:-2].split(' ')
    
def check_if_author(text):
    msg_split = text.lower().split(' ')
    for i in msg_split:
        lemma = morph.parse(i)[0].normal_form
        if lemma in authors_surnames:
            return lemma.title()
    return False

def get_name(text):
    name = ''
    doc = Doc(text.title())
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)
    for span in doc.spans:
        span.normalize(morph_vocab)
    for span in doc.spans:
        if span.type == 'PER':
            span.extract_fact(names_extractor)
            name = span.normal
    if name != '':
        return name
    else:
        if check_if_author(text) is False:
            return False
        else:
            return check_if_author(text)

In [35]:
with open(r'quotes_reading.txt') as file:
    quotes = file.read()
quotes = quotes[0:-1].split('\n')

def random_quote(text):
    return random.choice(quotes)

# Здесь изначально была идея генерировать цитату по набору цитат про чтение из интернета
# Но, кажется, пока собрала слишком маленький корус: markovify и самостоятельные попытки с биграммами выдают 
# слишком несвязный текст, а при работе с триграммами выдаются сами изначальные предложения

#     gentext=markovify.Text(quotes, state_size =1, well_formed=True)
#     return (gentext.make_short_sentence(80), gentext.make_short_sentence(80))

In [36]:
# По данному автору ищет похожих - используется https://www.literature-map.com
# Сайт англоязычный, но иногда русскоязычные имена принимает, а иногда нет - поэтому сначала пробую передать имя на русском,
# а в случае неудачи на всякий случай транслитерированное 

def similar_books(text):
    text = re.sub('\s', '+', text)
    url = f"https://www.literature-map.com/{text}"
    page = rq.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    try:
        soup.findAll("div", {'id':'gnodMap'})[0].text
        a = soup.findAll("div", {'id':'gnodMap'})[0].text
        authors = a.split('\n')[2:7]
        authors_out = str(authors[0])
        for i in authors[1:]:
            authors_out += f', {i}'
        return f'Обычно тем, кто нравится этот автор, также нравятся следующие: {authors_out}'
    except:
        text = translit(text, 'ru', reversed = True)
        text = re.sub('\s', '+', text)
        url = f"https://www.literature-map.com/{text}"
        page = rq.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        try:
            soup.findAll("div", {'id':'gnodMap'})[0].text
            a = soup.findAll("div", {'id':'gnodMap'})[0].text
            authors = a.split('\n')[2:7]
            authors_out = str(authors[0])
            for i in authors[1:]:
                authors_out += f', {i}'
            return f'Обычно тем, кто нравится этот автор, также нравятся следующие: {authors_out}'
        except:
            return False

## Сам бот

In [66]:
bot = telebot.TeleBot('5973691708:AAHafjUNwDv-9wcV28P9Gz3826pAz2OUcSY')
api_id = 'c4f85f9f56687c24d9bc1396dd40f79a'

city = ''
date = ''
43
@bot.message_handler(commands=['start'])
def send_welcome(message):
    
    bot.reply_to(message, "Привет, я телеграм-бот! Пока я умею говорить только про погоду и немного про книги, но я быстро учусь. Давай общаться!")
    bot.register_next_step_handler(message, get_intent)
    

@bot.message_handler(func=lambda m: True)
def get_intent(message):
    text = message.text
    if (get_city(text) != False or find_if_they_said_date(text) != False or get_coords(text) != False) and text.lower() not in fake_places:
        get_weather_info_from_message(message)
        
    else:    
        text_vec = vectorizer.transform([message.text])
        result = model.predict(text_vec)[0]
        intent = unique_classes[result]

        if intent == 'weather':
            get_weather_info_from_message(message)
        elif intent == 'literature':
            lit_choice(message)
        elif intent == 'goodbye':
            bot.send_message(message.from_user.id, text=random.choice(data_dict[intent]['responses']))
        else:
            bot.send_message(message.from_user.id, text=random.choice(data_dict[intent]['responses']))
            bot.send_message(message.from_user.id, 'Хочешь поговорить про погоду или книги?')

def get_weather_info_from_message(message):
    global city
    global date
    text = message.text
    if get_city(text) != False and find_if_they_said_date(text) != False:
        city = get_city(text)
        date = find_if_they_said_date(text)
        if date == "сейчас":
            bot.send_message(message.from_user.id, give_current_weather(city))
            
        else:
            bot.send_message(message.from_user.id, give_weather_for_date(city, date))
        city = ''
        date = ''
        bot.send_message(message.from_user.id, 'Что-то еще?')
            
    elif get_city(text) != False:
        city = get_city(text)
        date = ''
        bot.send_message(message.from_user.id, 'Я могу сказать, какая погода сейчас или какая погода будет в ближайшие четыре дня. Какой день тебя интересует?')
        bot.register_next_step_handler(message, register_when)

    elif find_if_they_said_date(text) != False:
        if get_coords(text) is False or text.lower() in fake_places:
            date = find_if_they_said_date(text)
            city = ''
            bot.send_message(message.from_user.id, 'Какой город тебя интересует?')
            bot.register_next_step_handler(message, register_where)
            return 'register_where'
        else:
            date = find_if_they_said_date(text)
            city = text
            if date == "сейчас":
                bot.send_message(message.from_user.id, give_current_weather(city))
            
            else:
                bot.send_message(message.from_user.id, give_weather_for_date(city, date))
            city = ''
            date = ''
            bot.send_message(message.from_user.id, 'Что-то еще?')
            
    else:

        if get_coords(text) is False or text.lower() in fake_places:
            date = ''
            city = ''
            bot.send_message(message.from_user.id, 'Какой город и день тебя интересует? Я могу сказать, какая погода сейчас или какая погода будет в ближайшие четыре дня.')
            bot.register_next_step_handler(message, get_weather_info_from_message)
        else:
            city = text
            date = ''
            bot.send_message(message.from_user.id, 'Я могу сказать, какая погода сейчас или какая погода будет в ближайшие четыре дня. Какой день тебя интересует?')
            bot.register_next_step_handler(message, register_when)


def register_when(message):
    global city, date
    dat = message.text
    
    if find_if_they_said_date(dat) == False:
        bot.send_message(message.from_user.id, 'Пока я предсказываю погоду только на ближайшие дни. Какой день тебя интересует?')
        date = ''
        bot.register_next_step_handler(message, register_when)
        
    else:
        date = find_if_they_said_date(dat)
        
        if date == "сейчас":
            bot.send_message(message.from_user.id, give_current_weather(city))
            
        else:
            bot.send_message(message.from_user.id, give_weather_for_date(city, date))
            
        city = ''
        date = ''
        bot.send_message(message.from_user.id, 'Что-то еще?')
        
def register_where(message):
    global city, date
    cit = message.text
    if get_city(cit) == False:
        if get_coords(cit) is False or cit.lower() in fake_places:
            bot.send_message(message.from_user.id, "Кажется, я такого не знаю - попробуй другой!")
            city = ''
            bot.register_next_step_handler(message, register_where)
        else:
            city = cit
            if date == "сейчас":
                bot.send_message(message.from_user.id, give_current_weather(city))
            else:
                bot.send_message(message.from_user.id, give_weather_for_date(city, date))
            city = ''
            date = ''
            bot.send_message(message.from_user.id, 'Что-то еще?')
    else:
        city = get_city(cit)
        if date == "сейчас":
            bot.send_message(message.from_user.id, give_current_weather(city))
        else:
            bot.send_message(message.from_user.id, give_weather_for_date(city, date))
        city = ''
        date = ''
        bot.send_message(message.from_user.id, 'Что-то еще?')

# Книги
def book_shop(message):
    book = get_book(message.text)
    if book != '':
        bot.send_message(message.from_user.id, get_book(message.text))
        bot.send_message(message.from_user.id, 'О чём еще хочешь поговорить?')
    else:
        bot.send_message(message.from_user.id, 'Что-то не могу найти такую книгу :(')
        bot.send_message(message.from_user.id, 'Давай поговорим про что-нибудь другое. Погода, книги?')
                         
def lit_choice(message):
    if get_name(message.text) != False:
        if similar_books(message.text) != False:
            bot.send_message(message.from_user.id, f'Тебе нравится {get_name(message.text)}? Мне тоже! {similar_books(message.text)}')

        else:
            bot.send_message(message.from_user.id, f'Тебе нравится {get_name(message.text)}? Мне тоже! Здорово, что у нас хороший вкус!\n\nВот несколько книг этого автора - надеюсь ты найдешь, что нибудь для себя!')
            bot.send_message(message.from_user.id,  get_book(message.text))
        
        bot.send_message(message.from_user.id, 'О чём еще хочешь поговорить?')


    else:
        what_next = random.choice(['ask_about_author', 'random_quote', 'suggest_to_buy'])
        if what_next == 'suggest_to_buy':
            bot.send_message(message.from_user.id, 'Я очень люблю книги и могу подсказать тебе, где их купить. Тебе это интересно?')
            bot.register_next_step_handler(message, get_yn_intent_about_buying)

        elif what_next == "random_quote":
            bot.send_message(message.from_user.id, random_quote(message.text))
            bot.send_message(message.from_user.id, 'О чём еще хочешь поговорить?')

        else:
            bot.send_message(message.from_user.id, 'Мои любимые авторы - Джонатан Франзен и Ричард Руссо. А какой твой любимый автор?')
            bot.register_next_step_handler(message, lit_choice)

def get_yn_intent_about_buying(message):
    text = message.text
    text_vec = yn_vectorizer.transform([text])
    result = yn_model.predict(text_vec)[0]
    intent = yn_unique_classes[result]

    if intent == 'yes':
        bot.send_message(message.from_user.id, 'Просто напиши мне, какую книгу ты хочешь купить!')
        bot.register_next_step_handler(message, book_shop)
    else:
        bot.send_message(message.from_user.id, 'Тогда давай поговорим про что-нибудь другое. Погода, книги?')
            
bot.polling(none_stop=True)